In [1]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

### Functions

In [2]:
def regression_model(feature, target, model_type="linear"):
    """
    Trains a regression model and returns the R² score.

    Parameters:
    df (DataFrame): The dataset containing features and target.
    target (str): The target variable name.
    model_type (str): "linear" for Linear Regression, "random_forest" for Random Forest.

    Returns:
    float: R² score of the model.
    """
    # Split features and target
    X = feature
    y = target
    
    # Get number of samples and features
    n = X.shape[0]  # Number of samples
    p = X.shape[1]  # Number of features

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
 

    # Select model
    if model_type == "linear":
        model = LinearRegression()
    elif model_type == "random_forest":
        model = RandomForestRegressor(n_estimators=100, random_state=42)
    elif model_type == "decision_tree":
        model = DecisionTreeRegressor(random_state=42)
    elif model_type == "lasso":
        model = Lasso(alpha=0.1, random_state=42)
    else:
        raise ValueError("Invalid model type. Choose 'linear' or 'random_forest'.")

    # Train the model
    model.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)

    # Compute Adjusted R²
    adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    
    return round(r2,4) , round(adjusted_r2,4), model

In [3]:
def f_regression_summary(feature, target, model):

    # Call f_regression once and unpack the result
    f_stats, p_values = f_regression(feature, target)
    
    # Create the summary dataframe
    reg_summary = pd.DataFrame(data=feature.columns.values, columns=['Features'])
    
    # Add coefficients from the model
    reg_summary['Coefficients'] = model[2].coef_
    
    # Add p-values
    reg_summary['p-values'] = p_values.round(3)
    
    return reg_summary

In [4]:
def feature_importance(feature, target, method="MDI"):

    # Split features and target
    X = feature
    y = target
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    if method == "MDI":
        # Get feature importances
        importances = rf_model.feature_importances_
        importances_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': importances
        }).sort_values(by='Importance', ascending=False)

    else :
        from sklearn.inspection import permutation_importance

        result = permutation_importance(rf_model, X_test, y_test, n_repeats=10, random_state=42)

        importances_df = pd.DataFrame({
            'Feature': X_test.columns,
            'Importance': result.importances_mean
        }).sort_values(by='Importance', ascending=False)
        

        
    return importances_df

## Import the data

In [5]:
import pandas as pd
df=pd.read_excel('input.xlsx', sheet_name='input_2024')
#df1=df1.drop(columns=['duplex','condo_fees_hyp'])
df1=df.drop(columns=['price_date','name','site','duplex','link','condo_fees_hyp','mortgage_monthly','heat_ref_monthly','total_month_fees','total_price'])
#drop lines with no data
df1.dropna(how='any', inplace=True)
df1.reset_index(drop=True, inplace=True)
df1.head()

,area,type,year,living_area,lot_area,floors,bedroom,bathroom,rooms_total,price,condo_fees_tax
0,verdun,condo,1919,335.0,335.0,1,1,1,4,289000,222.166667
1,verdun,condo,1987,680.0,680.0,1,3,1,4,325000,231.916667
2,verdun,condo,1984,820.0,820.0,1,2,1,5,329000,546.333333
3,verdun,condo,1984,775.0,775.0,1,2,1,5,330000,493.416667
4,ile des soeurs,condo,1994,586.0,586.0,1,1,1,4,348000,506.833333


In [6]:
df1.shape

(860, 11)

### Linear regression model

#### Create a dictionary of dataframes for each area

In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Create a scaler object
scaler = StandardScaler()

# Dictionary to store DataFrames and results
area_data = {}

# Get the full list of area/type dummy column names for consistency
all_area_types = pd.get_dummies(df1[['area', 'type']], dtype=int).columns

# Loop through each unique area
for area in df1['area'].unique():
    # Filter the DataFrame for the current area
    df_area = df1[df1['area'] == area]

    # Separate X (features), y (targets)
    X = df_area.drop(columns=['price', 'condo_fees_tax'])
    y = df_area['price'].values
    y1 = df_area['condo_fees_tax'].values

    # Split into numeric and categorical columns
    numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = ['area', 'type']

    # Scale numeric features
    X_numeric_scaled = scaler.fit_transform(X[numeric_cols])
    X_numeric_scaled_df = pd.DataFrame(X_numeric_scaled, columns=numeric_cols, index=X.index)  # Ensure index consistency

    # One-hot encode categorical columns
    X_categorical = pd.get_dummies(X[categorical_cols], dtype=int)

    # Reindex dummies to match full set of dummy columns (ensure consistent shape across areas)
    X_categorical = X_categorical.reindex(columns=all_area_types, fill_value=0, index=X.index)  # Ensure index consistency

    # Combine scaled numeric and binary dummy columns
    X_final_scaled = pd.concat([X_numeric_scaled_df, X_categorical], axis=1)

    # Store in dictionary
    area_data[area] = {
        'X': pd.concat([X[numeric_cols], X_categorical], axis=1),  # unscaled numeric + dummies
        'X_scaled': X_final_scaled,  # scaled numeric + binary dummies
        'y': y,
        'y1': y1
    }

# Now, you can safely use area_data for training linear regression, ensuring consistent indexes.


#### Create a dictionary to store scalers per area

In [8]:
# Define numeric columns to scale
numeric_columns = ['year', 'living_area', 'lot_area', 'floors', 'bedroom', 'bathroom', 'rooms_total']

# Dictionary to store scalers per area
area_scalers = {}

for area in df1['area'].unique():
    area_df = df1[df1['area'] == area].copy()
    X_area = pd.get_dummies(area_df.drop(columns=['price', 'condo_fees_tax']), columns=['area', 'type'], dtype=int)
    
    scaler = StandardScaler()
    scaler.fit(X_area[numeric_columns])
    
    area_scalers[area] = scaler

#### Linear regression for each area

In [9]:
linear_model = {}  # Initialize an empty dictionary to store models
linear_model_1 = {}

#create a dictionary of linear regression results for price per area
for area in df1['area'].unique():
    linear_model[f'{area}'] = regression_model(
        area_data[f'{area}']['X_scaled'],
        area_data[f'{area}']['y'],
        model_type="linear"
    )

#create a dictionary of linear regression results for condo_fees_tax per area
    linear_model_1[f'{area}'] = regression_model(
        area_data[f'{area}']['X_scaled'],
        area_data[f'{area}']['y1'],
        model_type="linear"        
    )


In [10]:
linear_model['laval_chomedy'][2].intercept_

374603.99959784816

In [11]:
linear_model['laval_chomedy'][2].coef_

array([  51293.46285517,  184058.03330589,   32239.19480134,
         -2231.11727411,   -6757.9670497 ,   40762.85989432,
        -19529.94976603,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,  179528.30151479, -179528.30151479])

#### Coefficients and p-values for price linear regression

In [12]:
from sklearn.feature_selection import f_regression

In [13]:
# Dictionary to store results
summary_dict = {}
for area in df1['area'].unique():
    X_summ = area_data[area]['X']
    y_summ = area_data[area]['y']
    summary=f_regression_summary(X_summ, y_summ, linear_model[area])
    # Store summary in dictionary
    summary_dict[area] = summary
    # Print summary for each area
    print(f"\n Condo Fees and tax F regression Summary for {area}:")
    print(summary)


 Condo Fees and tax F regression Summary for verdun:
                  Features  Coefficients  p-values
0                     year  26729.438734     0.029
1              living_area  52140.393636     0.000
2                 lot_area  -5877.078749     0.000
3                   floors  -7234.630298     0.000
4                  bedroom  52838.871608     0.000
5                 bathroom  39296.050803     0.000
6              rooms_total  38995.092572     0.000
7            area_brossard      0.000000     1.000
8      area_ile des soeurs      0.000000     1.000
9             area_lachine      0.000000     1.000
10            area_lasalle      0.000000     1.000
11      area_laval_chomedy      0.000000     1.000
12   area_laval_desrapides      0.000000     1.000
13     area_laval_duvernay      0.000000     1.000
14   area_laval_fabreville      0.000000     1.000
15     area_laval_pontviau      0.000000     1.000
16  area_laval_stedorothee      0.000000     1.000
17      area_saint_laurent  

#### Coefficients and p-values for condo fees and tax linear regression

In [14]:
# Dictionary to store results
summary_dict = {}
for area in df1['area'].unique():
    X_summ = area_data[area]['X']
    y1_summ = area_data[area]['y1']
    summary=f_regression_summary(X_summ, y1_summ, linear_model[area])
    # Store summary in dictionary
    summary_dict[area] = summary
    # Print summary for each area
    print(f"\n Condo Fees and tax F regression Summary for {area}:")
    print(summary)


 Condo Fees and tax F regression Summary for verdun:
                  Features  Coefficients  p-values
0                     year  26729.438734     0.105
1              living_area  52140.393636     0.455
2                 lot_area  -5877.078749     0.469
3                   floors  -7234.630298     0.155
4                  bedroom  52838.871608     0.504
5                 bathroom  39296.050803     0.980
6              rooms_total  38995.092572     0.740
7            area_brossard      0.000000     1.000
8      area_ile des soeurs      0.000000     1.000
9             area_lachine      0.000000     1.000
10            area_lasalle      0.000000     1.000
11      area_laval_chomedy      0.000000     1.000
12   area_laval_desrapides      0.000000     1.000
13     area_laval_duvernay      0.000000     1.000
14   area_laval_fabreville      0.000000     1.000
15     area_laval_pontviau      0.000000     1.000
16  area_laval_stedorothee      0.000000     1.000
17      area_saint_laurent  

#### OLS Regression on price table for each area

In [15]:
# OLS Regression models and adjusted R²
ols_model_price = {}
ols_model_fees = {}
ols_r2_price = {}
ols_r2_fees = {}

for area in df1['area'].unique():
    X_ols = sm.add_constant(area_data[area]['X'])
    
    # Price
    y_price = area_data[area]['y']
    model_price = sm.OLS(y_price, X_ols).fit()
    ols_model_price[area] = model_price
    ols_r2_price[area] = 1 - (1 - model_price.rsquared) * (len(y_price) - 1) / (len(y_price) - X_ols.shape[1] - 1)

    # Condo fees
    y_fees = area_data[area]['y1']
    model_fees = sm.OLS(y_fees, X_ols).fit()
    ols_model_fees[area] = model_fees
    ols_r2_fees[area] = 1 - (1 - model_fees.rsquared) * (len(y_fees) - 1) / (len(y_fees) - X_ols.shape[1] - 1)


### Random forest regression

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [17]:
#transform 'area' and 'type' to dummies variables
df_dummies = pd.get_dummies(df1, columns=['area','type'], dtype=int)
df_dummies.head()

,year,living_area,lot_area,floors,bedroom,bathroom,rooms_total,price,condo_fees_tax,area_brossard,...,area_laval_chomedy,area_laval_desrapides,area_laval_duvernay,area_laval_fabreville,area_laval_pontviau,area_laval_stedorothee,area_saint_laurent,area_verdun,type_condo,type_house
0,1919,335.0,335.0,1,1,1,4,289000,222.166667,0,...,0,0,0,0,0,0,0,1,1,0
1,1987,680.0,680.0,1,3,1,4,325000,231.916667,0,...,0,0,0,0,0,0,0,1,1,0
2,1984,820.0,820.0,1,2,1,5,329000,546.333333,0,...,0,0,0,0,0,0,0,1,1,0
3,1984,775.0,775.0,1,2,1,5,330000,493.416667,0,...,0,0,0,0,0,0,0,1,1,0
4,1994,586.0,586.0,1,1,1,4,348000,506.833333,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
X = df_dummies.drop(columns=['price','condo_fees_tax'])
y = df_dummies['price'].values
y_1= df_dummies['condo_fees_tax'].values
X.head()

,year,living_area,lot_area,floors,bedroom,bathroom,rooms_total,area_brossard,area_ile des soeurs,area_lachine,...,area_laval_chomedy,area_laval_desrapides,area_laval_duvernay,area_laval_fabreville,area_laval_pontviau,area_laval_stedorothee,area_saint_laurent,area_verdun,type_condo,type_house
0,1919,335.0,335.0,1,1,1,4,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,1987,680.0,680.0,1,3,1,4,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,1984,820.0,820.0,1,2,1,5,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,1984,775.0,775.0,1,2,1,5,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,1994,586.0,586.0,1,1,1,4,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
rf_reg_result = regression_model(X, y, model_type="random_forest")

In [20]:
rf_reg_result_1 = regression_model(X, y_1, model_type="random_forest")

In [21]:
rf_model = rf_reg_result[2]
rf_model_1 = rf_reg_result_1[2]

### Feature importanceDefault (MDI) - Using .feature_importances_

### Feature importance Permutation Importance

In [22]:
feature_imp_mdi = feature_importance(X, y, method="MDI")
feature_imp_mdi
feature_imp_perm = feature_importance(X, y, method="Perm")
feature_imp_perm

,Feature,Importance
1,living_area,0.439137
0,year,0.318495
5,bathroom,0.132430
2,lot_area,0.101315
6,rooms_total,0.036814
16,area_laval_stedorothee,0.024404
19,type_condo,0.019800
20,type_house,0.017971
3,floors,0.014197
18,area_verdun,0.005375


In [23]:
feature_imp_mdi = feature_importance(X, y_1, method="MDI")
feature_imp_mdi
feature_imp_perm = feature_importance(X, y_1, method="Perm")
feature_imp_perm

,Feature,Importance
4,bedroom,24494.130150
20,type_house,23980.558717
19,type_condo,11254.456106
9,area_lachine,4095.188386
13,area_laval_duvernay,2514.803988
8,area_ile des soeurs,2319.833628
14,area_laval_fabreville,1308.268393
12,area_laval_desrapides,400.036170
17,area_saint_laurent,-131.946024
15,area_laval_pontviau,-1366.207061


## Decision tree regression

In [24]:
from sklearn.tree import DecisionTreeRegressor

In [25]:
regression_model(X, y, model_type="decision_tree")

(0.8012, 0.7962, DecisionTreeRegressor(random_state=42))

In [26]:
regression_model(X, y_1, model_type="decision_tree")

(0.4992, 0.4866, DecisionTreeRegressor(random_state=42))

## Lasso regression

In [27]:
from sklearn.linear_model import Lasso

In [28]:
regression_model(X, y, model_type="lasso")

(0.5558, 0.5447, Lasso(alpha=0.1, random_state=42))

In [29]:
regression_model(X, y_1, model_type="lasso")

(0.5504, 0.5392, Lasso(alpha=0.1, random_state=42))

### Fill a form for a new entry

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd

# Example pre-trained models (Replace these with actual trained models)
rf_model = rf_reg_result[2]
rf_model_1 = rf_reg_result_1[2]


# Dropdown options
area_options = list(df1['area'].unique())

type_options = list(df1['type'].unique())

def validate_numeric(entry):
    """Check if the entry is a valid number."""
    try:
        float(entry)
        return True
    except ValueError:
        return False

def estimate_price():
    """Estimate price and condo fees using the trained models."""
    global data_df
    
    if rf_model is None or rf_model_1 is None:
        messagebox.showerror("Error", "Random Forest model not loaded")
        return
    
    # Get user entries
    entries = {
        "year": entry_year.get(),
        "living_area": entry_living.get(),
        "lot_area": entry_lot.get(),
        "floors": entry_floors.get(),
        "bedroom" : entry_bedrooms.get(),
        "bathroom": entry_bathrooms.get(),
        "rooms_total": entry_rooms.get(),
        "area": area_var.get(),
        "type": type_var.get()
    }

    # Validate numeric entries
    for field in ["year", "living_area", "lot_area", "floors","bedroom", "bathroom", "rooms_total"]:
        if not validate_numeric(entries[field]):
            messagebox.showerror("Error", f"Invalid input in {field}. Entry has to be a number.")
            return

    # Store the data
    data_df = pd.DataFrame([entries])
    
    # Create DataFrame for prediction
    input_df = pd.DataFrame([X.iloc[0]])  # Copy first row as a new DataFrame
    input_df.loc[:, :] = 0  # Ensure all values are initially zero

    for col in input_df.columns:
        if data_df['area'].iloc[0] in col:
            input_df.at[0, col] = 1  # Set the corresponding column to 1

    for col in input_df.columns:
        if data_df['type'].iloc[0] in col:
            input_df.at[0, col] = 1  # Set the corresponding column to 1

    for col in input_df.columns:
        if col in data_df.columns:
            input_df.loc[0, col] = data_df[col].iloc[0]

    area_selected = data_df['area'].iloc[0]

    
    # Scale numeric inputs for linear regression
    scaler = area_scalers.get(area_selected)
    if scaler:
        input_df_scaled = input_df.copy()
        input_df_scaled[numeric_columns] = scaler.transform(input_df_scaled[numeric_columns])
    else:
        input_df_scaled = input_df.copy()

    # OLS requires unscaled, one-hot encoded input with constant
    # Prepare input for OLS
    X_ols_input = input_df.copy()

    # Ensure all values are numeric
    X_ols_input = X_ols_input.apply(pd.to_numeric, errors='coerce')

    # Add constant term for OLS
    X_ols_input = sm.add_constant(X_ols_input, has_constant='add')

    # Align columns with the training model
    X_ols_input = X_ols_input[ols_model_price[area_selected].model.exog_names]


    # Predict price and condo fees using Random Forest
    estimated_price_rf = rf_model.predict(input_df.iloc[[0]])[0]
    estimated_condo_fees_rf = rf_model_1.predict(input_df.iloc[[0]])[0]

    # Predict price and condo fees using Linear Regression with input_df_lr
    estimated_price_lr = linear_model[area_selected][2].predict(input_df_scaled.iloc[[0]])[0]
    estimated_condo_fees_lr = linear_model_1[area_selected][2].predict(input_df_scaled.iloc[[0]])[0]

    # Predict with OLS models
    estimated_price_ols = ols_model_price[area_selected].predict(X_ols_input)[0]
    estimated_condo_fees_ols = ols_model_fees[area_selected].predict(X_ols_input)[0]



    # Retrieve Adjusted R² values for random forest regression
    adj_r2_price_rf = rf_reg_result[1]
    adj_r2_condo_fees_rf = rf_reg_result_1[1]
    
    # Retrieve Adjusted R² values for linear regression
    adj_r2_price_lr = linear_model[area_selected][1]
    adj_r2_condo_fees_lr = linear_model_1[area_selected][1]

    # Get Adjusted R² for OLS regression
    adj_r2_price_ols = ols_r2_price[area_selected]
    adj_r2_condo_fees_ols = ols_r2_fees[area_selected]


    label_rf_results.config(text=f"RF Price: {estimated_price_rf:,.0f} $ | Adj R²: {adj_r2_price_rf:.2f}\n"
                                 f"RF Condo Fees: {estimated_condo_fees_rf:,.0f} $ | Adj R²: {adj_r2_condo_fees_rf:.2f}")

    label_lr_results.config(text=f"LR Price: {estimated_price_lr:,.0f} $ | Adj R²: {adj_r2_price_lr:.2f}\n"
                                 f"LR Condo Fees: {estimated_condo_fees_lr:,.0f} $ | Adj R²: {adj_r2_condo_fees_lr:.2f}")
    
    label_ols_results.config(text=f"OLS Price: {estimated_price_ols:,.0f} $ | Adj R²: {adj_r2_price_ols:.2f}\n"
                              f"OLS Condo Fees: {estimated_condo_fees_ols:,.0f} $ | Adj R²: {adj_r2_condo_fees_ols:.2f}")



# Create main window
root = tk.Tk()
root.title("Property Estimator")

# Labels and Entry fields
tk.Label(root, text="Year:").grid(row=0, column=0)
entry_year = tk.Entry(root)
entry_year.grid(row=0, column=1)

tk.Label(root, text="Living Area (sq ft):").grid(row=1, column=0)
entry_living = tk.Entry(root)
entry_living.grid(row=1, column=1)

tk.Label(root, text="Lot Area (sq ft):").grid(row=2, column=0)
entry_lot = tk.Entry(root)
entry_lot.grid(row=2, column=1)

tk.Label(root, text="Number of Floors:").grid(row=3, column=0)
entry_floors = tk.Entry(root)
entry_floors.grid(row=3, column=1)

tk.Label(root, text="Number of Bedrooms:").grid(row=4, column=0)
entry_bedrooms = tk.Entry(root)
entry_bedrooms.grid(row=4, column=1)

tk.Label(root, text="Number of Bathrooms:").grid(row=5, column=0)
entry_bathrooms = tk.Entry(root)
entry_bathrooms.grid(row=5, column=1)

tk.Label(root, text="Total Number of Rooms:").grid(row=6, column=0)
entry_rooms = tk.Entry(root)
entry_rooms.grid(row=6, column=1)

# Dropdown for Area
tk.Label(root, text="Area:").grid(row=7, column=0)
area_var = tk.StringVar(value=area_options[0])
area_dropdown = ttk.Combobox(root, textvariable=area_var, values=area_options)
area_dropdown.grid(row=7, column=1)

# Dropdown for Type
tk.Label(root, text="Type:").grid(row=8, column=0)
type_var = tk.StringVar(value=type_options[0])
type_dropdown = ttk.Combobox(root, textvariable=type_var, values=type_options)
type_dropdown.grid(row=8, column=1)

# Estimate Button
btn_estimate = tk.Button(root, text="Estimate", command=estimate_price)
btn_estimate.grid(row=9, columnspan=2)


# Random Forest Results Section
tk.Label(root, text="Random Forest Results:").grid(row=12, column=0, columnspan=2)
label_rf_results = tk.Label(root, text="")
label_rf_results.grid(row=13, column=0, columnspan=2)

# Linear Regression Results Section
tk.Label(root, text="Linear Regression Results:").grid(row=14, column=0, columnspan=2)
label_lr_results = tk.Label(root, text="")
label_lr_results.grid(row=15, column=0, columnspan=2)

# OLS Regression Results Section
tk.Label(root, text="OLS Regression Results:").grid(row=16, column=0, columnspan=2)
label_ols_results = tk.Label(root, text="")
label_ols_results.grid(row=17, column=0, columnspan=2)



# Run the interface
root.mainloop()


C:\Users\PC\AppData\Local\Temp\ipykernel_16084\2706351063.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2000' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  input_df.loc[0, col] = data_df[col].iloc[0]
C:\Users\PC\AppData\Local\Temp\ipykernel_16084\2706351063.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1000' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  input_df.loc[0, col] = data_df[col].iloc[0]
C:\Users\PC\AppData\Local\Temp\ipykernel_16084\2706351063.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  input_df.loc[0, col] = data_df[col].iloc[0